# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
if 'uk_food' in databases:
    print("The 'uk_food' database was created successfully.")
else:
    print("The 'uk_food' database was not created.")

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()

In [ ]:
# review a document in the establishments collection
document = establishments_collection.find_one()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant",
    "RatingValue": None,  # This restaurant hasn't been rated yet, so set it to None
    "LocalAuthorityName": "Greenwich",  # Assuming Greenwich for this example
    "Latitude": "51.476",  # Example latitude, you should use the actual latitude
    "Longitude": "-0.009",  # Example longitude, you should use the actual longitude
    "Hygiene": None,  # Hygiene score not available yet
    "ConfidenceInManagement": None,  # Confidence score not available yet
    "Structural": None,  # Structural score not available yet
    "BusinessTypeID": None  # This will be updated based on the BusinessTypeID of "Restaurant/Cafe/Canteen"
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant_data)

In [ ]:
# Check that the new restaurant was inserted
query = {"BusinessName": "Penang Flavours"}
new_restaurant = establishments.find_one(query)

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = establishments.find_one(query, projection)


from pprint import pprint
pprint(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID

query = {"Name": "Penang Flavours"}

update = {"$set": {"BusinessTypeID": business_type_id}}

result = establishments.update_one(query, update)

if result.matched_count > 0:
    print("Penang Flavours restaurant updated with correct BusinessTypeID.")
else:
    print("No restaurant found with the name 'Penang Flavours'.")

In [ ]:
# Confirm that the new restaurant was updated

penang_flavours = establishments.find_one({"Name": "Penang Flavours"})

from pprint import pprint
pprint(penang_flavours)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"

dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

print("Number of documents in Dover:", dover_count)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"

result = establishments.delete_many({"LocalAuthorityName": "Dover"})

print(f"Number of documents deleted: {result.deleted_count}")

In [ ]:
# Check if any remaining documents include Dover

count_dover = establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of remaining documents with Dover: {count_dover}")

if count_dover > 0:
    result = establishments.find_one({"LocalAuthorityName": "Dover"})
    pprint(result)
    

In [ ]:
# Check that other documents remain with 'find_one'

result = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})

pprint(result)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude

update_result = establishments.update_many(
    {"latitude": {"$type": "string"}},  # Find documents where latitude is a string
    [{"$set": {"latitude": {"$toDouble": "$latitude"}}}]  # Convert latitude to decimal (float)
)

update_result = establishments.update_many(
    {"longitude": {"$type": "string"}},  # Find documents where longitude is a string
    [{"$set": {"longitude": {"$toDouble": "$longitude"}}}]  # Convert longitude to decimal (float)
)

print(update_result.modified_count, "documents updated")

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue

update_result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},  # Find documents where RatingValue is a string
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]  # Convert RatingValue to integer
)

print(update_result.modified_count, "documents updated")

In [ ]:
# Check that the coordinates and rating value are now numbers

document = establishments.find_one()

print("Latitude:", document['latitude'], type(document['latitude']))
print("Longitude:", document['longitude'], type(document['longitude']))
print("RatingValue:", document['RatingValue'], type(document['RatingValue']))